In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns

from visu_utils import *

import re

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
EXPORT_PATH_TABLE = Path("../../papers/Distillation-MI-ICLR/tables/nlp/")
EXPORT_PATH_FIG = Path("../../papers/Distillation-MI-ICLR/figures/nlp/")

# Make sure the folders exist
EXPORT_PATH_TABLE.mkdir(parents=True, exist_ok=True)
EXPORT_PATH_FIG.mkdir(parents=True, exist_ok=True)


MTEB_BASELINES_PATH = Path("../non_sync/baselines_mteb/mteb_detailed/en_Classification.csv")

RESULTS_PATHS = [Path("../non_sync/mteb_benchmarking/results/experiments_gist_nll"),
                 Path("../non_sync/mteb_benchmarking/results/experiments_gist_mse")
                 ]



df_merged = load_classification_merged_mteb(MTEB_BASELINES_PATH, RESULTS_PATHS)

# all but Student

df_merged = df_merged[~df_merged["Model"].str.contains('Student')]


In [3]:
base_students_models = df_merged[df_merged["Model"].str.contains('snowflake') & ~df_merged["Model"].str.contains('-l')].copy()
base_students_models = base_students_models[~base_students_models['Model'].str.contains('1.5')]
# /lustre/fsn1/projects/rech/ehz/uwf24rf/EMIR/nlp_embeddings/embeddings_gist/jamesgpt1/sf_model_e5  /lustre/fsn1/projects/rech/ehz/uwf24rf/EMIR/nlp_embeddings/embeddings_gist/WhereIsAI/UAE-Large-V1          /lustre/fsn1/projects/rech/ehz/uwf24rf/EMIR/nlp_embeddings/embeddings_gist/Salesforce/SFR-Embedding-2_R /lustre/fsn1/projects/rech/ehz/uwf24rf/EMIR/nlp_embeddings/embeddings_gist/dunzhang/stella_en_400M_v5

teachers = ["jamesgpt1/sf_model_e5", "WhereIsAI/UAE-Large-V1", "Salesforce/SFR-Embedding-2_R", "dunzhang/stella_en_400M_v5"]


teacher_models = df_merged[df_merged["Model"].isin(teachers)].copy()




teacher_models[" "] = "Teacher"
base_students_models[" "] = "Student (Base)"


In [4]:


merged = pd.concat([teacher_models, base_students_models])

merged = merged.set_index([' ', 'Model']).drop(['loss', 'Training step', 'Dataset'], axis=1)
display(merged)

df_pivot = merged.rename(
        {
            "Model Size (Million Parameters)": "Size",
        },
        axis=1)

# latex_results['Task'] = latex_results['Task'].apply(lambda x: x.replace("Classification", ""))
# latex_results['Task'] = latex_results['Task'].apply(lambda x: re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x))
# latex_results['Task'] = latex_results['Task'].apply(lambda x: x.replace(" (en)", ""))
# latex_results['Task'] = latex_results['Task'].apply(lambda x: x.replace(" ", " \\\\ "))
# latex_results['Task'] = latex_results['Task'].apply(lambda x: f"\\rotatebox{{90}}{{\\shortstack{{{x}}}}}")

# average all colums but size
avg = df_pivot[[c for c in df_pivot.columns if c != "Size"]].mean(axis=1)

df_pivot["Avg."] = avg

df_pivot.index = pd.MultiIndex.from_tuples([(x[0],x[1].split("/")[-1]) for x in df_pivot.index])


# rename columns
df_pivot.columns = df_pivot.columns.str.replace("Classification", "")
df_pivot.columns = [re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x) for x in df_pivot.columns]
df_pivot.columns = df_pivot.columns.str.replace(" (en)", "")
df_pivot.columns = df_pivot.columns.str.replace(" ", " \\\\ ")
df_pivot.columns = [f"\\rotatebox{{90}}{{\\shortstack{{{x}}}}}" for x in df_pivot.columns]
style = df_pivot.style.format("{:.1f}")
# style = style.format("{:.0f}", subset=['Size'])
style = style.format_index(escape="latex")

latex = style.to_latex(
    clines="skip-last;data",
    hrules=True,
    sparse_index=True,
    multicol_align="c",
    multirow_align="c",
    caption="Performance of the 4 teachers we used and of the base students. Experiments with single teacher distillation were performed with the stronger teacher SFR-Embedding-2\\_R.",
    label=f"tab:nlp:base_student_teacher_perfs",
    column_format="llc|" + "c" * (len(df_pivot.columns) - 2) + "|c",
)

# add resizebox
latex = latex.replace(
    "\\begin{tabular}",
    "\\resizebox{\\textwidth}{!}{\\begin{tabular}",
)
latex = latex.replace(
    "\\end{tabular}",
    "\\end{tabular}}\n",
)

with open(EXPORT_PATH_TABLE / "table_base_student_teacher_perfs.tex", "w") as f:
    f.write(latex)

Model Size (Million Parameters)  \
               Model                                                                  
Teacher        Salesforce/SFR-Embedding-2_R                                  7111.0   
               dunzhang/stella_en_400M_v5                                     435.0   
               WhereIsAI/UAE-Large-V1                                         335.0   
               jamesgpt1/sf_model_e5                                          335.0   
Student (Base) Snowflake/snowflake-arctic-embed-m                             109.0   
               Snowflake/snowflake-arctic-embed-s                              33.0   
               Snowflake/snowflake-arctic-embed-xs                             23.0   

                                                    AmazonCounterfactualClassification (en)  \
               Model                                                                          
Teacher        Salesforce/SFR-Embedding-2_R                                           92.72   
               dunzhang/stella_en_400M_v5                                             92.36   
               WhereIsAI/UAE-Large-V1                                                 75.55   
               jamesgpt1/sf_model_e5                                                  70.85   
Student (Base) Snowflake/snowflake-arctic-embed-m                                     76.81   
               Snowflake/snowflake-arctic-embed-s                                     71.18   
               Snowflake/snowflake-arctic-embed-xs                                    65.09   

                                                    AmazonPolarityClassification  \
               Model                                                               
Teacher        Salesforce/SFR-Embedding-2_R                                97.31   
               dunzhang/stella_en_400M_v5                                  97.19   
               WhereIsAI/UAE-Large-V1                                      92.84   
               jamesgpt1/sf_model_e5                                       91.81   
Student (Base) Snowflake/snowflake-arctic-embed-m                          82.83   
               Snowflake/snowflake-arctic-embed-s                          78.75   
               Snowflake/snowflake-arctic-embed-xs                         70.04   

                                                    AmazonReviewsClassification (en)  \
               Model                                                                   
Teacher        Salesforce/SFR-Embedding-2_R                                    61.04   
               dunzhang/stella_en_400M_v5                                      59.53   
               WhereIsAI/UAE-Large-V1                                          48.29   
               jamesgpt1/sf_model_e5                                           48.94   
Student (Base) Snowflake/snowflake-arctic-embed-m                              38.93   
               Snowflake/snowflake-arctic-embed-s                              38.26   
               Snowflake/snowflake-arctic-embed-xs                             35.34   

                                                    Banking77Classification  \
               Model                                                          
Teacher        Salesforce/SFR-Embedding-2_R                           90.02   
               dunzhang/stella_en_400M_v5                             89.30   
               WhereIsAI/UAE-Large-V1                                 87.69   
               jamesgpt1/sf_model_e5                                  84.61   
Student (Base) Snowflake/snowflake-arctic-embed-m                     80.34   
               Snowflake/snowflake-arctic-embed-s                     79.12   
               Snowflake/snowflake-arctic-embed-xs                    76.37   

                                                    EmotionClassification  \
               Model                                                     